## Data cleaning
In this notebook, we will clean the data and prepare it for the analysis.

### Import libraries

In [4]:
import gpxpy
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # ignore warnings

Load the data.

In [5]:
# load data
df = pd.read_csv('starting_data.csv')
df.head()

,_id,length_3d,user,start_time,max_elevation,bounds,uphill,moving_time,end_time,max_speed,gpx,difficulty,min_elevation,url,downhill,name,length_2d
0,5afb229e8f80884aaad9c6ea,10832.953016,Bergfritz,2018-05-11 07:37:40,1934.47,"{'min': {'type': 'Point', 'coordinates': [13.2...",612.880,12155.0,2018-05-11 11:38:23,1.595493,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,1322.96,http://www.hikr.org/tour/post131855.html,609.670,"Remsteinkopf, 1945 m",10832.953016
1,5afb229e8f80884aaad9c6eb,12259.376315,Bergfritz,2018-05-12 07:25:08,2186.21,"{'min': {'type': 'Point', 'coordinates': [13.1...",614.753,13876.0,2018-05-12 12:08:28,1.394320,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T3 - Difficult Mountain hike,1266.40,http://www.hikr.org/tour/post131856.html,1193.733,"Schuhflicker, 2214 m",12259.376315
2,5afb229e8f80884aaad9c6ec,22980.168081,igor,2018-05-11 06:29:38,2265.00,"{'min': {'type': 'Point', 'coordinates': [8.99...",2255.976,28971.0,2018-05-11 15:32:43,1.503002,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T3 - Difficult Mountain hike,176.54,http://www.hikr.org/tour/post131839.html,2177.626,Cima d'erbea Est quota 2164m e Gaggio 2267m,22980.168081
3,5afb229e8f80884aaad9c6ed,24903.503470,rkroebl,2018-05-10 07:06:22,962.42,"{'min': {'type': 'Point', 'coordinates': [8.43...",882.312,26726.0,2018-05-10 16:15:18,1.516689,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,388.51,http://www.hikr.org/tour/post131840.html,901.052,Waldstätterweg: Alpnachstad - Buochs,24903.503470
4,5afb229e8f80884aaad9c6ee,19581.273819,rkroebl,2018-05-11 05:44:58,697.57,"{'min': {'type': 'Point', 'coordinates': [8.61...",310.662,18197.0,2018-05-11 12:54:25,1.542405,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...",T2 - Mountain hike,438.50,http://www.hikr.org/tour/post131845.html,305.372,Waldstätterweg: Buochs - Beckenried und Gersau...,19581.273819


Considering that, According to a New York Times article on “speed hiking”, most hikers average speed is about 2 mph (miles per hour). 

We can therefor drop:
- Data that have a average speed higher than 2.5m/s (~ 5.5mph). 
<!-- - Data that have a average speed lower than 0.3m/s (~ 0.6mph). -->
- Not available data.
- Data that have a duration lower than 30 minutes or higher than 14 hours.
- Data that have a length lower than 0.5km or higher than 25km.
- Duplicate data.

In [6]:
df['avg_speed'] = df['length_3d']/df['moving_time']
df['difficulty_num'] = df['difficulty'].map(lambda x: int(x[1])).astype('int32')

In [7]:
temp_length = len(df)

# drop na values
df.dropna()
mid_length = len(df)
print("Number of rows after dropping NA values : " + str(temp_length - mid_length))

# drop avg speed values
df = df[df['avg_speed'] < 2.5] # an avg of > 2.5m/s is probably not a hiking activity
#df = df[df['avg_speed'] > 0.3] # an avg of < 0.3m/s is probably not a hiking activity --> they could stop
temp_length = len(df)
print("Number of rows after dropping AVERAGE SPEED values : " + str(mid_length - temp_length))

# drop duration values
df = df[df['moving_time'] > (30*60)] # not consider durations < 30min
df = df[df['moving_time'] < (14*60*60)] # not consider durations > 14h
mid_length = len(df)
print("Number of rows after dropping DURATION values : " + str(temp_length - mid_length))

# drop length values
df = df[df['length_3d'] > 1000] # not consider lengths < 1000m
df = df[df['length_3d'] < 20000] # not consider lengths > 20km
temp_length = len(df)
print("Number of rows after dropping LENGTH values : " + str(mid_length - temp_length))

# drop duplicate values
df = df.drop_duplicates(subset=['name', 'difficulty', 'length_3d', 'moving_time', 'avg_speed', 'difficulty_num'])
mid_length = len(df)
print("Number of rows after dropping DUPLICATE values : " + str(temp_length - mid_length))


print("Final number of rows : " + str(len(df)))

Number of rows after dropping NA values : 0
Number of rows after dropping AVERAGE SPEED values : 2718
Number of rows after dropping DURATION values : 145
Number of rows after dropping LENGTH values : 1274
Number of rows after dropping DUPLICATE values : 1986
Final number of rows : 6018


## Prepare data for modelling

In [8]:
from geopy.geocoders import Nominatim 

geolocator = Nominatim(user_agent="geospatial course unitn")

def get_country(row):
    try:
        gpx = gpxpy.parse(row['gpx'])
        segment = gpx.tracks[0].segments[0]

        data = []
        for point_idx, point in enumerate(segment.points):
            data.append([point.longitude, point.latitude,
                        point.elevation, point.time])

        start_point = data[0]

        latlon = str(start_point[1]) + "," + str(start_point[0])
        starting_place = geolocator.reverse(latlon)
        
        row['country'] = starting_place.raw['address']['country']

        return row
    except:
        row['country'] = 'Unknown'
        return row

In [9]:
df = df.apply(get_country, axis=1)

In [10]:
df['country'] = df['country'].replace(['Schweiz/Suisse/Svizzera/Svizra'], 'Switzerland')
df['country'].value_counts()

Switzerland                                  2222
Italia                                       2179
Österreich                                    546
Deutschland                                   475
France                                        176
Unknown                                       113
España                                         56
United States                                  36
Liechtenstein                                  25
Norge                                          21
United Kingdom                                 18
Ελλάς                                          18
Türkiye                                        12
Kosova / Kosovo                                12
Éire / Ireland                                 10
Ísland                                          8
България                                        8
საქართველო                                      6
Shqipëria                                       6
Северна Македонија                              5


In [11]:
# filter the countries, keep just "Italia", "Switzerland", "France", "Deutschland", "Österreich"
df = df[df['country'].isin(['Italia', 'Switzerland', 'France', 'Deutschland', 'Österreich'])]

In [12]:
df.to_csv('df_cleaned.csv', index=False)